In [16]:
!pip install gdown

# --- 1. ADIM: VERİ SETİNİ KAGGLE ORTAMINA İNDİRME VE AÇMA ---
import os

# İndirilecek dosyanın Google Drive ID'si ve hedef dosya adı
# Link: https://drive.google.com/file/d/1bPtSXPnMYP5z3PUUVoZPk7NLfm8R_mzM/
file_id = "1bPtSXPnMYP5z3PUUVoZPk7NLfm8R_mzM"
zip_dosya_yolu = "MURA-v1.1.zip"
hedef_klasor = "unziped_mura/" # Dosyaların çıkarılacağı klasör

# gdown ile dosyayı indiriyoruz
print("Veri seti indiriliyor...")
!gdown --id {file_id} -O {zip_dosya_yolu}
print("\nİndirme tamamlandı.")

# Hedef klasörü hazırlama ve zip dosyasını açma
print(f"\n'{hedef_klasor}' hazırlanıyor...")
!rm -rf "{hedef_klasor}" # Eğer klasör varsa temizle
!mkdir -p "{hedef_klasor}"
!unzip -q -n "{zip_dosya_yolu}" -d "{hedef_klasor}" # -q (quiet) modu logları azaltır, -n (no overwrite)
print("\nZip dosyasından çıkarma işlemi tamamlandı.")

Veri seti indiriliyor...
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1bPtSXPnMYP5z3PUUVoZPk7NLfm8R_mzM
From (redirected): https://drive.google.com/uc?id=1bPtSXPnMYP5z3PUUVoZPk7NLfm8R_mzM&confirm=t&uuid=b7517ffe-0622-4a90-84b9-a324bcc16b37
To: /kaggle/working/MURA-v1.1.zip
100%|███████████████████████████████████████| 3.38G/3.38G [00:30<00:00, 110MB/s]

İndirme tamamlandı.

'unziped_mura/' hazırlanıyor...

Zip dosyasından çıkarma işlemi tamamlandı.


In [18]:
# --- 2. ADIM: VERİLERİ YÜKLEME VE HAZIRLAMA ---
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

main_dir = hedef_klasor # Artık ana dizinimiz burası
train_csv_path = os.path.join(main_dir, "MURA-v1.1/train_image_paths.csv")
valid_csv_path = os.path.join(main_dir, "MURA-v1.1/valid_image_paths.csv")

# Veri çerçevelerini oluşturma
train_df = pd.read_csv(train_csv_path, header=None, names=['image_path'])
train_df['full_path'] = train_df['image_path'].apply(lambda path: os.path.join(main_dir, path))
train_df['label'] = train_df['image_path'].apply(lambda path: 1 if 'positive' in path else 0)
train_df['label'] = train_df['label'].astype(str)

valid_df = pd.read_csv(valid_csv_path, header=None, names=['image_path'])
valid_df['full_path'] = valid_df['image_path'].apply(lambda path: os.path.join(main_dir, path))
valid_df['label'] = valid_df['image_path'].apply(lambda path: 1 if 'positive' in path else 0)
valid_df['label'] = valid_df['label'].astype(str)

print("\nEğitim Veri Seti Dağılımı:\n", train_df['label'].value_counts())
print("\nDoğrulama Veri Seti Dağılımı:\n", valid_df['label'].value_counts())



Eğitim Veri Seti Dağılımı:
 label
0    21935
1    14873
Name: count, dtype: int64

Doğrulama Veri Seti Dağılımı:
 label
0    1667
1    1530
Name: count, dtype: int64


In [19]:
# --- 3. ADIM: SINIF AĞIRLIKLARINI HESAPLAMA ---
labels_for_weights = train_df['label'].astype(int).values
class_weights = compute_class_weight('balanced', classes=np.unique(labels_for_weights), y=labels_for_weights)
class_weight_dict = dict(enumerate(class_weights))
print(f"\nHesaplanan Sınıf Ağırlıkları: {class_weight_dict}")


Hesaplanan Sınıf Ağırlıkları: {0: 0.8390243902439024, 1: 1.2374100719424461}


In [20]:
# --- 4. ADIM: VERİ ÜRETEÇLERİ VE ZENGİNLEŞTİRME ---
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

print("\nVeri Üreteçleri Oluşturuluyor...")
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='full_path',
    y_col='label',
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    class_mode='binary',
    batch_size=BATCH_SIZE,
    shuffle=True
)

validation_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid_df,
    x_col='full_path',
    y_col='label',
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    class_mode='binary',
    batch_size=BATCH_SIZE,
    shuffle=False
)



Veri Üreteçleri Oluşturuluyor...
Found 36808 validated image filenames belonging to 2 classes.
Found 3197 validated image filenames belonging to 2 classes.


In [21]:
# --- 5. ADIM: TRANSFER ÖĞRENME MODELİNİ KURMA (ÖZELLİK ÇIKARIM) ---
base_model = ResNet50V2(input_shape=(*IMAGE_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])
print("\n--- Model Özeti (Özellik Çıkarım Aşaması) ---")
model.summary()



--- Model Özeti (Özellik Çıkarım Aşaması) ---


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50v2 (Functional)         │ (None, 7, 7, 2048)     │    23,564,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,614,401 (93.90 MB)

 Trainable params: 1,049,601 (4.00 MB)

 Non-trainable params: 23,564,800 (89.89 MB)

In [ ]:
# --- 6. ADIM: İLK EĞİTİM (SADECE ÜST KATMANLAR) ---
INITIAL_EPOCHS = 20
early_stopping_initial = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

print(f"\nModel {INITIAL_EPOCHS} epoch boyunca (sadece üst katmanlar) eğitiliyor...")
history = model.fit(
    train_generator,
    epochs=INITIAL_EPOCHS,
    validation_data=validation_generator,
    class_weight=class_weight_dict,
    callbacks=[early_stopping_initial]
)


Model 20 epoch boyunca (sadece üst katmanlar) eğitiliyor...
Epoch 1/20
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 561s 465ms/step - accuracy: 0.6920 - loss: 0.5828 - val_accuracy: 0.7557 - val_loss: 0.5054
Epoch 2/20
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 513s 446ms/step - accuracy: 0.7586 - loss: 0.5113 - val_accuracy: 0.7776 - val_loss: 0.4876
Epoch 3/20
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 515s 447ms/step - accuracy: 0.7849 - loss: 0.4809 - val_accuracy: 0.7873 - val_loss: 0.4786
Epoch 4/20
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 515s 448ms/step - accuracy: 0.7971 - loss: 0.4593 - val_accuracy: 0.7976 - val_loss: 0.4678
Epoch 5/20
1151/1151 ━━━━━━━━━━━━━━━━━━━━ 516s 448ms/step - accuracy: 0.8051 - loss: 0.4394 - val_accuracy: 0.7923 - val_loss: 0.4721
Epoch 6/20
  98/1151 ━━━━━━━━━━━━━━━━━━━━ 7:46 443ms/step - accuracy: 0.8213 - loss: 0.4252

In [23]:
# --- 7. ADIM: İNCE AYAR (FINE-TUNING) AŞAMASI ---
print("\n--- İnce Ayar (Fine-Tuning) Aşaması Başlıyor ---")
base_model.trainable = True
fine_tune_at = 140
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
print("\n--- Model Özeti (İnce Ayar Aşaması) ---")
model.summary()



--- İnce Ayar (Fine-Tuning) Aşaması Başlıyor ---

--- Model Özeti (İnce Ayar Aşaması) ---


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50v2 (Functional)         │ (None, 7, 7, 2048)     │    23,564,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,614,401 (93.90 MB)

 Trainable params: 17,401,857 (66.38 MB)

 Non-trainable params: 7,212,544 (27.51 MB)

In [24]:
# --- 8. ADIM: İNCE AYAR EĞİTİMİ ---
FINE_TUNE_EPOCHS = 30
TOTAL_EPOCHS = INITIAL_EPOCHS + FINE_TUNE_EPOCHS

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-7, verbose=1)
early_stopping_fine_tune = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)

# initial_epoch'u, önceki eğitimin bittiği yerden başlatıyoruz
# Eğer ilk eğitim erken durduysa, history.epoch[-1] + 1 doğru başlangıç noktasını verir
last_epoch = len(history.epoch)

print(f"\nModel {FINE_TUNE_EPOCHS} epoch daha (ince ayar) eğitiliyor...")
history_fine_tune = model.fit(
    train_generator,
    epochs=last_epoch + FINE_TUNE_EPOCHS, # Toplam epoch sayısına göre ayarla
    initial_epoch=last_epoch, # Kaldığı yerden devam et
    validation_data=validation_generator,
    class_weight=class_weight_dict,
    callbacks=[reduce_lr, early_stopping_fine_tune]
)

NameError: name 'history' is not defined

In [ ]:
# --- 9. ADIM: NİHAİ DEĞERLENDİRME VE GÖRSELLEŞTİRME ---
print("\nİnce ayar sonrası nihai model performansı değerlendiriliyor...")
scores = model.evaluate(validation_generator)
print("-" * 50)
print(f"İNCE AYAR SONRASI NİHAİ DOĞRULUK: {scores[1] * 100:.2f}%")
print(f"İNCE AYAR SONRASI NİHAİ KAYIP: {scores[0]}")
print("-" * 50)

# Eğitim geçmişlerini birleştirme
acc = history.history['accuracy'] + history_fine_tune.history.get('accuracy', [])
val_acc = history.history['val_accuracy'] + history_fine_tune.history.get('val_accuracy', [])
loss = history.history['loss'] + history_fine_tune.history.get('loss', [])
val_loss = history.history['val_loss'] + history_fine_tune.history.get('val_loss', [])

# Grafiği çizdirme
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Eğitim Doğruluğu')
plt.plot(val_acc, label='Doğrulama Doğruluğu')
plt.axvline(last_epoch - 1, color='gray', linestyle='--', label='İnce Ayar Başlangıcı')
plt.legend(loc='best')
plt.title('Toplam Eğitim ve Doğrulama Doğruluğu')
plt.xlabel('Epoch')
plt.ylabel('Doğruluk')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Eğitim Kaybı')
plt.plot(val_loss, label='Doğrulama Kaybı')
plt.axvline(last_epoch - 1, color='gray', linestyle='--', label='İnce Ayar Başlangıcı')
plt.legend(loc='best')
plt.title('Toplam Eğitim ve Doğrulama Kaybı')
plt.xlabel('Epoch')
plt.ylabel('Kayıp')

plt.tight_layout()
plt.show()
